In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.text import text_to_word_sequence

from transformers import AutoConfig, AutoModel,AutoTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [24]:
df2=pd.read_csv('../raw_data/starter_title_train.csv')

In [25]:
df=pd.read_csv('../raw_data/lunch3.csv')

In [26]:
df.head(2)

,Unnamed: 0,id,channel_id,country,channel_viewCount,channel_subscriberCount,channel_videoCount,title,published,views
0,0,IXfLYUTZhJU,UCYeRWGx2ej3YUvrRdzLugcQ,US,400579,1300,23,The City [Full Tape] (Atlanta Views &amp; Scen...,2022-10-10T12:00:11Z,6356
1,0,4VlLf3mQ4Nk,UCUR_q3i5yONr9BghspQzqyg,NaN,4297315883,6590000,2728,The Voice All Stars | EP.12 Semi Final เดือดอย...,2022-10-10T09:00:01Z,1278078


In [27]:
len(df)-578

6053

In [28]:
df=df[df.id.duplicated()==False]

In [29]:
#Getting rid of non-english
from langdetect import detect
df_english_titles=df[df['title'].apply(lambda x:detect(x))=='en']

In [30]:
df_english_titles

,Unnamed: 0,id,channel_id,country,channel_viewCount,channel_subscriberCount,channel_videoCount,title,published,views
0,0,IXfLYUTZhJU,UCYeRWGx2ej3YUvrRdzLugcQ,US,400579,1300,23,The City [Full Tape] (Atlanta Views &amp; Scen...,2022-10-10T12:00:11Z,6356
2,0,0SLOacCb6YY,UCiKIYazpPHGLK93Jilwj3kQ,NaN,39356,351,101,Hack the Box - PhotoBomb - Walkthrough,2022-10-10T01:14:43Z,0
3,0,VOmks_Unu1U,UCzYfz8uibvnB7Yc1LjePi4g,US,14572543681,14900000,4124,100 Days as the LETTER F in Minecraft!,2022-10-09T19:15:00Z,3570087
4,0,naEz3XJ11Q8,UCYC3WWrqFERLEnE9jPkRMvw,US,59707773,394000,147,"The Child That Was Born Twice, EP 2 | roblox b...",2022-10-09T15:00:13Z,1091742
7,0,1umZzyLaTQQ,UCb1oLJAwjQWFcuR-1JlcNHg,GB,5134585,586000,353,How to Unlock the POWER PUNCHER in Bonelab,2022-10-08T19:20:08Z,1123
...,...,...,...,...,...,...,...,...,...,...
6624,0,mn8w5vbLcm8,UCOXYyN5z9_vgLulI6hzJAUw,NaN,67179,44,54,Never Be The Same Again,2022-10-07T12:48:24Z,103
6626,0,h1lpwd41Cwg,UCSf-NCzjwcnXErUBW_qeFvA,US,2181115984,4650000,513,Mismo Dios (Same God) | Letras Oficiales | Ele...,2022-10-07T04:00:27Z,168338
6627,0,QLXZG5474_k,UCCCg8Fh5Avkt0lqD4wnAL6A,NO,51061189,170000,119,&quot;We eat and train the same&quot;,2022-10-06T16:35:21Z,739920
6628,0,TLV91fTgjVY,UC-ikaAXJqrepGbTymwDURsg,US,24719812,102000,189,Do you have the same struggle?! (Tool link In ...,2022-10-06T01:12:51Z,21915


In [31]:
import re
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer


def preprocessing(sentence):
    #stripping emojis
    def deEmojify(text):
        regrex_pattern = re.compile(pattern = "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags = re.UNICODE)
        return regrex_pattern.sub(r'',text)
    sentence=deEmojify(sentence)
    
    #stripping punctuations
    for char in string.punctuation:
        sentence=sentence.replace(char,'')
        
    #stripping chinese punctuations
    sentence=re.sub(u'(?<=[^0-9])[^\u4e00-\u9fff0-9a-zA-Z]+(?=[^0-9])',' ',sentence)
    
    #turning lowercase
    sentence=''.join(re.findall(r'\D',(sentence.strip()).lower()))
    
    #tokenizing words
    sentence=word_tokenize(sentence)
    
    #lemmatizing words
    sentence=[WordNetLemmatizer().lemmatize(w) for w in sentence]

    
    return sentence



In [32]:
#apply preprocessing
# df['clean_english_text']=df_english_titles['title'].apply(lambda x:preprocessing(x))



In [33]:
df_english_titles['title']=df_english_titles['title'].apply(lambda x:preprocessing(x))

/var/folders/7r/9jv1lnvj4b7b5xwwyx68yrwc0000gn/T/ipykernel_60376/4254019188.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english_titles['title']=df_english_titles['title'].apply(lambda x:preprocessing(x))


In [34]:
# df_english_titles=df_english_titles.dropna() 

In [35]:
df_english_titles

,Unnamed: 0,id,channel_id,country,channel_viewCount,channel_subscriberCount,channel_videoCount,title,published,views
0,0,IXfLYUTZhJU,UCYeRWGx2ej3YUvrRdzLugcQ,US,400579,1300,23,"[the, city, full, tape, atlanta, view, amp, sc...",2022-10-10T12:00:11Z,6356
2,0,0SLOacCb6YY,UCiKIYazpPHGLK93Jilwj3kQ,NaN,39356,351,101,"[hack, the, box, photobomb, walkthrough]",2022-10-10T01:14:43Z,0
3,0,VOmks_Unu1U,UCzYfz8uibvnB7Yc1LjePi4g,US,14572543681,14900000,4124,"[day, a, the, letter, f, in, minecraft]",2022-10-09T19:15:00Z,3570087
4,0,naEz3XJ11Q8,UCYC3WWrqFERLEnE9jPkRMvw,US,59707773,394000,147,"[the, child, that, wa, born, twice, ep, roblox...",2022-10-09T15:00:13Z,1091742
7,0,1umZzyLaTQQ,UCb1oLJAwjQWFcuR-1JlcNHg,GB,5134585,586000,353,"[how, to, unlock, the, power, puncher, in, bon...",2022-10-08T19:20:08Z,1123
...,...,...,...,...,...,...,...,...,...,...
6624,0,mn8w5vbLcm8,UCOXYyN5z9_vgLulI6hzJAUw,NaN,67179,44,54,"[never, be, the, same, again]",2022-10-07T12:48:24Z,103
6626,0,h1lpwd41Cwg,UCSf-NCzjwcnXErUBW_qeFvA,US,2181115984,4650000,513,"[mismo, dios, same, god, letras, oficiales, el...",2022-10-07T04:00:27Z,168338
6627,0,QLXZG5474_k,UCCCg8Fh5Avkt0lqD4wnAL6A,NO,51061189,170000,119,"[quotwe, eat, and, train, the, samequot]",2022-10-06T16:35:21Z,739920
6628,0,TLV91fTgjVY,UC-ikaAXJqrepGbTymwDURsg,US,24719812,102000,189,"[do, you, have, the, same, struggle, tool, lin...",2022-10-06T01:12:51Z,21915


In [36]:
X = df_english_titles['title']
y = df_english_titles['views']

In [37]:
len(X)

4507

In [38]:
# X=df_eng['clean_english_text'].dropna()
# y=df_eng.set_index('id')['views']

### baseline


In [42]:
y.mean()

2649572.2642556024

In [43]:
baseline2=(abs(y-y.mean())).mean()
baseline2

3816012.276794685

In [44]:
# X_train_tok.apply(lambda x:len(x)).max()

In [45]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


tk=Tokenizer()
tk.fit_on_texts(X)

X_tok = tk.texts_to_sequences(X)
X_pad=pad_sequences(X_tok, padding='post', dtype='float32')


# X_train_tok = tk.texts_to_sequences(X_train)

# X_test_tok = tk.texts_to_sequences(X_test)

# X_train_pad=pad_sequences(X_train_tok, padding='post', dtype='float32')

# X_test_pad=pad_sequences(X_test_tok, padding='post', dtype='float32')

In [46]:
X_tok

[[1, 235, 89, 2188, 3430, 524, 14, 3431, 3432, 525],
 [261, 1, 1589, 3433, 1237],
 [47, 3, 1, 589, 663, 4, 85],
 [1, 292, 34, 72, 1238, 1590, 145, 204, 1239, 2189],
 [16, 2, 997, 1, 262, 3434, 4, 367],
 [1, 3435, 109, 10, 198, 1240, 8],
 [1, 998, 22, 34, 999, 1, 1241, 1000, 1, 95, 9],
 [3436, 590, 664, 248, 1, 351, 31, 2190, 352],
 [3437, 79, 3438, 1, 590, 1001, 293, 3439],
 [2191, 23, 1242, 2192, 12, 1, 3440, 1, 1002, 109, 45, 89, 45, 3441],
 [1243, 127, 526, 527, 1, 1003, 15, 3, 305, 2193, 215, 3442],
 [1004, 4, 115, 1, 3443, 216, 1244, 483, 89, 45, 8, 11, 163, 12, 31],
 [1, 591, 2194, 10, 8],
 [1245, 207, 448, 74, 17, 1, 3444, 145],
 [1591, 738, 1, 274, 9, 1, 174, 2195, 3445, 528, 2, 262, 1, 79],
 [2196, 4, 1, 3446, 327, 294, 79, 328, 484, 45, 294, 79, 328, 484],
 [1, 3447, 1, 2197, 862, 11, 3448, 1592, 1, 589, 3449, 2198, 295],
 [146, 152, 739, 55, 1, 3450, 665, 1593, 2199],
 [2200, 3451, 296, 1, 351, 40, 312, 352],
 [3452, 590, 664, 1, 351, 31, 2190, 352],
 [2201,
  1005,
  1,
  1

In [47]:
vocab_size = len(tk.word_index)
vocab_size

9251

In [48]:
np.unique(np.array(y)).shape

(4033,)

In [49]:
df_english_titles.id.duplicated().sum()

0

In [50]:
len(np.unique(np.array(df['id'])))

5663

In [51]:
input_length=np.array([len(i) for i in X]).max()

In [52]:
from tensorflow.keras.layers import Normalization
from tensorflow.keras import Sequential,layers

embedding_size=100
from tensorflow.keras.layers import Dense, Flatten, SimpleRNN, LSTM, GRU,Dropout




def initialize_model():
    model=Sequential()
    model.add(layers.Embedding(
        input_dim=vocab_size+1, # 16 +1 for the 0 padding
        input_length=input_length, # Max_sentence_length (optional, for model summary)
        output_dim=embedding_size, # 100
        mask_zero=True, # Built-in masking layer :)
    ))

    
    model.add(layers.Conv1D(20, kernel_size=3))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation='linear'))
    
    model.compile(
        loss='mse',
        optimizer='adam',
        metrics='mae'
    ) 
   
    return model

In [53]:
from tensorflow.keras.callbacks import EarlyStopping
model=initialize_model()
es=EarlyStopping(patience=50, restore_best_weights=True)
history=model.fit(X_pad,y,batch_size=16,validation_split=0.2,callbacks=es,epochs=1000)



Epoch 1/1000


/Users/sonnykong/.pyenv/versions/3.10.6/envs/youtube_optimizer/lib/python3.10/site-packages/keras/engine/data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]
2022-11-29 17:10:01.419853: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


226/226 [==============================] - 1s 5ms/step - loss: 128437708652544.0000 - mae: 3305775.2500 - val_loss: 128079137603584.0000 - val_mae: 3798690.7500
Epoch 2/1000
226/226 [==============================] - 1s 4ms/step - loss: 122722046705664.0000 - mae: 3898507.5000 - val_loss: 127242516561920.0000 - val_mae: 3830131.7500
Epoch 3/1000
226/226 [==============================] - 1s 5ms/step - loss: 112923103985664.0000 - mae: 3574853.2500 - val_loss: 127591029669888.0000 - val_mae: 3181913.5000
Epoch 4/1000
226/226 [==============================] - 1s 5ms/step - loss: 90569359491072.0000 - mae: 2928354.2500 - val_loss: 148004455579648.0000 - val_mae: 3388402.7500
Epoch 5/1000
226/226 [==============================] - 1s 6ms/step - loss: 70932089536512.0000 - mae: 2479060.5000 - val_loss: 162764714672128.0000 - val_mae: 3550597.7500
Epoch 6/1000
226/226 [==============================] - 1s 6ms/step - loss: 54163450888192.0000 - mae: 2212623.2500 - val_loss: 165227509317632.0

226/226 [==============================] - 1s 5ms/step - loss: 8062800232448.0000 - mae: 998355.8750 - val_loss: 301154449752064.0000 - val_mae: 5677959.5000
Epoch 49/1000
226/226 [==============================] - 1s 6ms/step - loss: 14093711835136.0000 - mae: 1079323.7500 - val_loss: 341453591019520.0000 - val_mae: 6056023.0000
Epoch 50/1000
226/226 [==============================] - 1s 5ms/step - loss: 13237372321792.0000 - mae: 1034488.5000 - val_loss: 298247226654720.0000 - val_mae: 5755348.5000
Epoch 51/1000
226/226 [==============================] - 1s 6ms/step - loss: 12221703782400.0000 - mae: 993737.9375 - val_loss: 310381146800128.0000 - val_mae: 5828187.5000
Epoch 52/1000
226/226 [==============================] - 1s 6ms/step - loss: 13591040229376.0000 - mae: 1036947.1250 - val_loss: 366215318994944.0000 - val_mae: 6129623.0000


In [56]:
mae2=min(history.history['val_mae'])

In [57]:
mae2

3161922.25

In [60]:
(baseline2-mae2)/baseline2

0.17140668827829283

In [71]:
baseline1=4745395.994626171

In [70]:
((baseline1-1818918.75)/baseline1)

0.6166982161109845

In [ ]:
first time = 90%

In [ ]:
10415108.0000

In [ ]:
7201640.5000

In [ ]:
84298.0000
8,279,018.0000

In [ ]:
831485.0000
3123700
365622
8,365,622.5000

In [17]:
# import torch
# from transformers import BertTokenizer, BertModel

# # OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
# import logging
# #logging.basicConfig(level=logging.INFO)

# import matplotlib.pyplot as plt


# # Load pre-trained model tokenizer (vocabulary)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [18]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [20]:
# df_bert_vec=df['title'].apply(lambda x:tokenizer.tokenize(x))

In [21]:
# df_bert_vec

0      [[UNK], [UNK], 新, [UNK], [UNK], hd, 24, 小, [UN...
1      [crisp, ##y, french, fries, recipe, !, you, ca...
2      [[UNK], live, web, ##cam, from, lan, ##zar, ##...
3      [so, we, got, band, ##ico, ##ot, -, nap, ##ped...
4      [n, ., gin, has, made, his, own, an, ##ima, ##...
                             ...                        
739                        [big, update, tomorrow, ?, !]
740                   [magnetic, anti, slip, drill, bit]
741    [al, ##gui, ##en, ha, cobra, ##do, por, es, ##...
742    [ice, ##fr, ##og, doesn, &, #, 39, ;, t, want,...
743    [gt, ##a, 5, online, [UNK], mega, ramp, ##e, p...
Name: title, Length: 744, dtype: object